In [10]:
#from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications.vgg16 import VGG16, preprocess_input



In [11]:

HEIGHT = 300
WIDTH = 300

base_model = VGG16(weights='imagenet', 
                      include_top=False, 
                      input_shape=(HEIGHT, WIDTH, 3))



base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 300, 300, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 300, 300, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 300, 300, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 150, 150, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 150, 150, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 150, 150, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 75, 75, 128)       0     

In [12]:
from keras.preprocessing.image import ImageDataGenerator

TRAIN_DIR = "/Volumes/My Passport for Mac/data/images/"
BATCH_SIZE = 8

train_datagen =  ImageDataGenerator(
      preprocessing_function=preprocess_input,
      rotation_range=90,
      horizontal_flip=True,
      vertical_flip=True
    )

train_generator = train_datagen.flow_from_directory(TRAIN_DIR, 
                                                    target_size=(HEIGHT, WIDTH), 
                                                    batch_size=BATCH_SIZE)

Found 20580 images belonging to 120 classes.


In [13]:
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Sequential, Model

def build_finetune_model(base_model, dropout, fc_layers, num_classes):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    for fc in fc_layers:
        # New FC layer, random init
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(dropout)(x)

    # New softmax layer
    predictions = Dense(num_classes, activation='softmax')(x) 
    
    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    return finetune_model

class_list = ["dogs", "cats", "horses"]
FC_LAYERS = [1024, 1024]
dropout = 0.5

finetune_model = build_finetune_model(base_model, 
                                      dropout=dropout, 
                                      fc_layers=FC_LAYERS,
                                      num_classes = 120)
                                      # /*num_classes=len(class_list))*/

In [15]:
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

NUM_EPOCHS = 10
BATCH_SIZE = 10
num_train_images = 20580

adam = Adam(lr=0.00001)
finetune_model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])

filepath="/Volumes/My Passport for Mac/model/vgg16_model_weights.h5"
checkpoint = ModelCheckpoint(filepath, monitor=["acc"], verbose=0, mode='max')
callbacks_list = [checkpoint]

history = finetune_model.fit_generator(train_generator, epochs=NUM_EPOCHS,  
                                       steps_per_epoch=num_train_images // BATCH_SIZE, 
                                       shuffle=True, callbacks=callbacks_list)



Epoch 1/10
2058/2058 [==============================] - 3014s 1s/step - loss: 4.9343 - accuracy: 0.0115
Epoch 2/10
2058/2058 [==============================] - 2947s 1s/step - loss: 4.8191 - accuracy: 0.0128
Epoch 3/10
2058/2058 [==============================] - 2804s 1s/step - loss: 4.8027 - accuracy: 0.0117
Epoch 4/10
2058/2058 [==============================] - 2823s 1s/step - loss: 4.7967 - accuracy: 0.0132
Epoch 5/10
2058/2058 [==============================] - 2829s 1s/step - loss: 4.7830 - accuracy: 0.0136
Epoch 6/10
2058/2058 [==============================] - 2830s 1s/step - loss: 4.7636 - accuracy: 0.0197
Epoch 7/10
2058/2058 [==============================] - 2836s 1s/step - loss: 4.7397 - accuracy: 0.0244
Epoch 8/10
2058/2058 [==============================] - 2839s 1s/step - loss: 4.7233 - accuracy: 0.0257
Epoch 9/10
  41/2058 [..............................] - ETA: 48:00 - loss: 4.6883 - accuracy: 0.0244

KeyboardInterrupt: 

In [ ]:

plot_training(history)

# Plot the training and validation loss + accuracy
def plot_training(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))

    plt.plot(epochs, acc, 'r.')
    plt.plot(epochs, val_acc, 'r')
    plt.title('Training and validation accuracy')

    # plt.figure()
    # plt.plot(epochs, loss, 'r.')
    # plt.plot(epochs, val_loss, 'r-')
    # plt.title('Training and validation loss')
    plt.show()

    plt.savefig('acc_vs_epochs.png')